In [23]:
!pip install ml-datasets
# !python -m spacy download en_core_web_md

  Using cached spacy-3.0.0.tar.gz (7.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [497 lines of output]
        Obtaining dependency information for setuptools from https://files.pythonhosted.org/packages/bb/26/7945080113158354380a12ce26873dd6c1ebd88d47f5bc24e2c5bb38c16a/setuptools-68.2.2-py3-none-any.whl.metadata
        Using cached setuptools-68.2.2-py3-none-any.whl.metadata (6.3 kB)
        Obtaining dependency information for cython>=0.25 from https://files.pythonhosted.org/packages/f0/a7/42116e4be098b5ae75669b76ad62216e2f67c5a9b8f87d6aa2b99bc9f9d7/Cython-3.0.3-cp312-cp312-win_amd64.whl.metadata
        Using cached Cython-3.0.3-cp312-cp312-win_amd64.whl.metadata (3.2 kB)
        Obtaining dependency information for cymem<2.1.0,>=2.0.2 from https://files.pythonhosted.org/packages/35/e0/34b11adc80502f0760ce2892dfdfcd8a7f450acd3147156c98620cb4071d/cymem-2.0.8-cp312-cp312-win_amd64.whl.metadata
        Using cached cymem-2.0.8-

  Using cached ml_datasets-0.2.0-py3-none-any.whl (15 kB)
  Obtaining dependency information for srsly<3.0.0,>=1.0.1 from https://files.pythonhosted.org/packages/06/b4/d620235df9104c9049c5378027fb2692a8a51fafc775e2feae815ff99599/srsly-2.4.8-cp312-cp312-win_amd64.whl.metadata
  Using cached srsly-2.4.8-cp312-cp312-win_amd64.whl.metadata (20 kB)
Using cached srsly-2.4.8-cp312-cp312-win_amd64.whl (478 kB)


In [24]:
!python -m spacy download ru_core_news_sm

✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


You should consider upgrading via the 'c:\Users\emely\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [51]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
import pandas as pd
from sklearn.model_selection import train_test_split

In [52]:
nlp = spacy.load("ru_core_news_sm")

c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'ru_core_news_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


загрузка дата сета

In [53]:
df = pd.read_csv("./women-clothing-accessories.3-class.balanced.csv", delimiter='\t')
# фильтруем создаем новый DataFrame, который содержит только те строки из df, 
df = df = df[df['sentiment'] != 'neautral']
# поделили
train_data, test_data = train_test_split(df.values, test_size=0.2, random_state=42)

nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [54]:
def make_docs(data):
    docs = []
    # nlp.pipe([texts]) намного быстрее, чем запуск
    # nlp(text) для каждого текста
    # as_tuples позволяет нам передавать кортеж,
    # первый из которых обрабатывается как текст
    # второй будет возвращен таким, какой он есть.
    for doc, label in tqdm(nlp.pipe(data, n_process=8, as_tuples=True), total = len(data)):
        # print(label)
        if label == 'neg' or label == 'negative':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [55]:
# num_texts = 100 # берем 100 текстов
train_docs = make_docs(train_data)
valid_docs = make_docs(test_data)

100%|██████████| 12000/12000 [00:56<00:00, 212.49it/s] 


In [56]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

https://spacy.io/usage/training#quickstart

base_config.cfg

https://spacy.io/usage/training#quickstart
from scipy.spatial.distance import cosine

# что здесь происходит: мы заполнили base_config, а эта команда на его основе прописывает config.cfg

! python -m spacy init fill-config base_config.cfg config.cfg

! python -m spacy train config.cfg --output ./output

In [57]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [58]:
# ! python -m spacy train config.cfg --output ./output
!python -m spacy train config.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'textcat', 'tagger']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TEXTCAT  LOSS TAGGER  CATS_SCORE  TAG_ACC  SCORE 
---  ------  ------------  ------------  -----------  ----------  -------  ------
  0       0          0.00          0.25        69.46       51.47    35.62    0.44
  0     200        163.06         36.65      5552.84       84.90    89.53    0.87
  0     400        182.72         24.21      2599.07       88.15    93.30    0.91
  0     600        166.79         20.75      2361.49       89.14    94.24    0.92
  0     800        168.51         18.97      2536.09       90.01    94.91    0.92
  0    1000        185.17         15.56      2839.11       90.38    95.18    0.93
  0    1200        206.33         15.49  

Cгенерировал модели

In [61]:
nlp = spacy.load("output/model-best")
text=""

reviews = [
    'Отличный ресторан! Обслуживание на высшем уровне, официанты приветливые и внимательные', # +
    'Ужасное место! Обслуживание медленное и невнимательное. Еда была холодной, и она вообще не соответствовала описанию в меню', # -
    'Этот смартфон просто потрясающий! Быстрая производительность, яркий и четкий экран, долгое время работы от аккумулятора',
    'Настоящий литературный шедевр!',
    'Честно говоря, эта книга - отстой. Сюжет предсказуемый, персонажи плоские и скучные', # -
    'Плохо сделано, не возможно пользоваться' # -
    'В челом все хорошо, но есть не болшие минусы в сюжете. Ставлю 4' # +
]

for text in reviews:
    doc = nlp(text)
    if doc.cats['positive'] > .5:
        print('Позитивный отзыв')
    else:
        print('Негативный отзыв')



Позитивный отзыв
Негативный отзыв
Позитивный отзыв
Позитивный отзыв
Негативный отзыв
Позитивный отзыв
